In [2]:
print("ok")

ok


In [3]:
%pwd

'c:\\Users\\Shanthaswetha\\OneDrive\\Documents\\something_new\\Medical-chatbot\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Shanthaswetha\\OneDrive\\Documents\\something_new\\Medical-chatbot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract dsata from the pdf file

def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_file(data = 'Data/')

In [9]:
#extracted_data

In [10]:
#split the data into text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size =500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("Length of the text chunks", len(text_chunks))

Length of the text chunks 6973


In [12]:
#text_chunks

In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [14]:
#download the embedding from Hugging face

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\Shanthaswetha\AppData\Local\Temp\ipykernel_5632\4114186597.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
#query_result

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY=os.environ.get('GEMINI_API_KEY')

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

KeyboardInterrupt: 

In [21]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY


In [23]:
#embed each chunk and upsert the embeddings into your Pinecone index.

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

KeyboardInterrupt: 

In [24]:
#Load existing index 

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [25]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs={"k":3})

In [27]:
retrieved_docs = retriever.invoke("what is Acne?")

In [28]:
retrieved_docs

[Document(id='2dd6a415-0662-40b0-a3f3-40f46a94f8a6', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='17c4e974-b015-4a76-8daa-11b2ef5aaf2b', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # or "gemini-1.5-pro" if you want higher quality
    temperature=0.4,
    max_output_tokens=500,
    google_api_key=GEMINI_API_KEY
)


In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt =(
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieval context to answer "
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer conncise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human", "{input}")
    ]
)

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_cahin = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_cahin.invoke({"input": "what is headache?"})
print(response["answer"])

Headaches are pain in the head, often accompanied by symptoms like nausea, vomiting, and light sensitivity.  Tension-type headaches are the most common, causing dull pressure.  More severe headaches, like cluster headaches, involve intense one-sided pain.
